# Get Started

<a target="_blank" href="https://colab.research.google.com/github/numqi/numqi/blob/main/docs/foundation/get_started/get_started.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This `get-started` is for new users from optimization perspective. For users with quantum information background, you may start with `Application/get-started`.

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch

try:
    import numqi
except ImportError:
    %pip install numqi
    import numqi

import numqi

np_rng = np.random.default_rng()

## Unconstrained optimization

A general unconstrained optimization problem can be formulated as

$$ \min_{x\in\mathbb{R}^n} f(x) $$

where $f(x)$ is the objective function, $x$ is the optimization variable, and $\mathbb{R}^n$ is n-dimension Euclidean space, also known as the search space.

A specific example is the Rosenbrock function [wiki-link](https://en.wikipedia.org/wiki/Rosenbrock_function)

$$ f(x)=\sum_{i=1}^{n-1}[100(x_{i+1}-x_i^2)^2 + (1-x_i)^2] $$

whose global minimum is at $x=(1,1,\cdots,1)$ and $f(x)=0$.

$$ f(1,\cdots,1)=0 $$

In [ ]:
class Rosenbrock(torch.nn.Module):
    def __init__(self, n):
        super().__init__()
        self.theta = torch.nn.Parameter(torch.tensor(np_rng.uniform(-1, 1, size=n), dtype=torch.float64))

    def forward(self):
        tmp0 = self.theta[1:] - self.theta[:-1]**2
        tmp1 = 1-self.theta[:-1]
        ret = 100*torch.dot(tmp0, tmp0) + torch.dot(tmp1,tmp1)
        return ret


Let's first verify that $(1,\cdots,1)$ gives global minimum of the Rosenbrock function.

In [ ]:
n = 4
model = Rosenbrock(n)
hf0 = numqi.optimize.hf_model_wrapper(model)
x_optim = np.ones(n)
print('f(1,...,1)=', hf0(x_optim, tag_grad=False))

Then, let's try to find the optimal point from random initial points.

In [ ]:
theta_optim = numqi.optimize.minimize(model, theta0='uniform', num_repeat=3, tol=1e-10)
print('optimal x:', theta_optim.x)
print('optimal value:', theta_optim.fun)

Above, `num_repeat=3` will run the optimization 3 times with different initial points, in hope of finding the global minimum. Under the hood, we call `scipy.optimize.minimize` using `method='L-BFGS-B'`, and the gradient is obtained from pytorch's backward pass.

To make it clear, we are NOT saying `numqi` have solved the problem completely (there is no guarantee that the global minimum is found). Even worse, when the dimension is large, the global minimum is hard to find and the convergence might be bad as shown below. `numqi` provides a simple interface for users to start with, and we encourage users to explore more advanced optimization techniques.

In [ ]:
model = Rosenbrock(10)
theta_optim = numqi.optimize.minimize(model, theta0='uniform', num_repeat=3, tol=1e-10)
print('optimal x:', theta_optim.x)

## Manifold optimization

`numqi` is designed to address manifold optimization, one kind of constrained optimization, which can be formulated as

$$ \min_{x\in\mathcal{M}} f(x) $$

where $\mathcal{M}$ is a manifold, a subset of $\mathbb{R}^n$ with some constraints. For example, the n-Sphere manifold $S^n$ is the set of points in $\mathbb{R}^{n+1}$ that are at a fixed distance from the origin,

$$ S^n = \{x\in\mathbb{R}^{n+1} : \|x\|_2=1\} $$

PS: Mathemtically, topology is reqiured to clearly define a manifold, which is far beyond the scope of this document (also the author's knowledge).

`numqi` adopts the trivialization strategy from [arxiv-link](https://arxiv.org/abs/1909.09501), converting manifold optimization to unconstrained optimization:

$$ \min_{x\in\mathbb{R}^n} f(\phi(x)) $$

where $\phi(x)$ is a mapping from $\mathbb{R}^n$ to $\mathcal{M}$.

PS: we strongly recommend these two papers for mathematical details about trivialization strategy. `numqi` borrows the idea from them and apply it to quantum information problems.

1. [arxiv-link](https://arxiv.org/abs/1909.09501) Trivializations for Gradient-Based Optimization on Manifolds
2. [arxiv-link](https://arxiv.org/abs/2203.04794) Geometric Optimisation on Manifolds with Applications to Deep Learning

### Mushroom function

PS: Mushroom is one of the famous HKUST's sea view landmark, see [hkust-link](https://hkust.edu.hk/multimedia/gallery/atrium), that's why we name this function as Mushroom.

$$ \min_{(x,y)\in S^1} f(x,y)=x^4+y^4+2x^2y^3 $$

It's some polynomial function defined on the 1-Sphere manifold $S^1$. The numerical optimal solution is around

$$x^*=\pm 0.67226298, y^*=-0.74031242, f(x^*,y^*)=0.1378840$$

First, we can visualize the function on the 2D plane. (please skip reading the following looooong code block which just for a better visualization)


In [ ]:
cp_tableau = ['#4c72b0', '#dd8452', '#55a868', '#c44e52', '#8172b3', '#937860', '#da8bc3', '#8c8c8c', '#ccb974', '#64b5cd']
theta_list = np.linspace(0, 2*np.pi, 100)
xdata = np.cos(theta_list)
ydata = np.sin(theta_list)
hf0 = lambda x,y: x**4 + y**4 + 2*x**2*y**3 + 1 #add constant 1 for visualization
theta_optim = -2.308057679636519 #from optimization later

text_kw = dict(verticalalignment='center', horizontalalignment='center', fontsize=16)
arrow_kw = dict(arrowstyle="Simple, tail_width=0.5, head_width=4, head_length=8", color='k')
fig,ax = plt.subplots(figsize=(6,5.2))
ax.plot(xdata, ydata, color=cp_tableau[0])
ax.text(0.3, 1.15, '$(x,y)$', **text_kw, color=cp_tableau[0])
ax.text(-0.5, 0, 'circle $S^1$', **text_kw, color=cp_tableau[0])
y0 = hf0(0, 1)
ax.text(0.35, y0+0.15, '$f(x,y)$', **text_kw, color=cp_tableau[1])
ax.plot([0], [1], '.', color=cp_tableau[0], markersize=6)
ax.plot([0], [y0], '.', color=cp_tableau[1], markersize=6)
ax.add_patch(matplotlib.patches.FancyArrowPatch((0,1.05), (0, y0-0.05), **arrow_kw))
fval = hf0(xdata, ydata)
ax.plot(xdata*fval, ydata*fval, color=cp_tableau[1])
r0 = hf0(np.cos(theta_optim), np.sin(theta_optim))
x0 = r0*np.cos(theta_optim)
y0 = r0*np.sin(theta_optim)
ax.plot([x0,-x0], [y0,y0], 'o', markersize=6, color='red')
tmp0 = dict(text_kw)
tmp0['fontsize'] = 16
ax.text(-x0+0.4, y0-0.15, 'optimal', **tmp0, color='red')
ax.set_ylim(-2.5, 2.5)
ax.set_aspect('equal')
fig.tight_layout()


Above, each point on the circle $(x,y)\in S^1$ is mapped to $f(x,y)$ (orange-colored curve, looks like a mushroom). The optimal solution is highlighted by the red dot. From the plot, we can see that the red dot is indeed the global minimum.

Then, let's use `numqi` to find the optimal solution in three steps

1. define the manifold in `__init__()`
2. implement the objective function in `forward()`
3. run the minimization

In [ ]:
class Mushroom(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # define the manifold
        self.manifold = numqi.manifold.Sphere(2, dtype=torch.float64)

    def forward(self):
        # implement the objective function
        x,y = self.manifold()
        loss = x**4 + y**4 + 2*x**2*y**3
        return loss

# run the minimization
model = Mushroom()
theta_optim = numqi.optimize.minimize(model, theta0='uniform', num_repeat=3, tol=1e-10, print_every_round=0)

# print the results
x,y = model.manifold().detach().numpy()
print(x, y, theta_optim.fun)

Luckily, you may find the optimal solution in the first run (if not, try more runs).

### Minimum eigen vector

The minimum eigen vector problem is to find the eigenvector corresponding to the smallest eigenvalue of a given matrix $A$.

$$ \min_{x\in S^{n-1}} f(x)=x^T A x $$

where $S^{n-1}$ is the n-Sphere manifold $S^{n-1}=\{x\in\mathbb{R}^n : \|x\|_2=1\}$.

In [ ]:
class DummyMinEigen(torch.nn.Module):
    def __init__(self, mat):
        super().__init__()
        self.mat = torch.tensor(mat, dtype=torch.float64)
        self.manifold = numqi.manifold.Sphere(mat.shape[0], dtype=torch.float64)

    def forward(self):
        vec = self.manifold()
        loss = torch.dot(vec, (self.mat @ vec))
        return loss


Below, we generate a random symmetric matrix $A$ and find the minimum eigen vector using `numqi`.

In [ ]:
N0 = 128
tmp0 = np_rng.normal(size=(N0,N0))
mat = (tmp0 + tmp0.T) / 2

model = DummyMinEigen(mat)
theta_optim = numqi.optimize.minimize(model, theta0='uniform', num_repeat=3, tol=1e-14, print_every_round=0)
EVL = theta_optim.fun
EVC = model.manifold().detach().numpy()
EVL_ = np.linalg.eigvalsh(mat)[0]
print('error(EVL)', np.abs(EVL-EVL_))
print('mae(EVC)', np.abs(mat @ EVC - EVC * EVL).max())
